In [1]:
from htc_numpy import HTC_Numpy
import numpy as np

params = {
'cav_dim' : 2,
'vib_dim' : 2,
'n_qubits' : 2,
'n_vib' : 2, # 1 vibrational mode per qubit
'g' : 0.1,
'omega_cav' : 1.0,
'omega_vib' : 0.1,
'omega_qubit' : 1.0,
'lambda_vib' : 0.1,
}

# Example: 1 cavity, 2 qubits, 2 vib modes, truncation=2
model = HTC_Numpy(params)

print("Hilbert space dimension:", model.dim)   # 32
print("First 5 labels:", model.labels[:5])

# Get a basis vector
psi = model.basis_state(0)
print("Basis[0] =", model.labels[0], psi.T)



Total Hilbert space dimension: 32
coupling strength g: 0.1
Hilbert space dimension: 32
First 5 labels: ['|0, g, 0, g, 0>', '|0, g, 0, g, 1>', '|0, g, 0, e, 0>', '|0, g, 0, e, 1>', '|0, g, 1, g, 0>']
Basis[0] = |0, g, 0, g, 0> [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [4]:
# build subspace operators

# bosonic operators
lowering = model.annihilation(model.cav_dim)
raising = model.creation(model.cav_dim)

# qubit operators
sigma_plus = model.sigma_plus()
sigma_minus = model.sigma_minus()
sigma_z = model.sigma_z()


# embed operators into full hilbert space
am = model.embed_operator(lowering, position=0)   # position=0 is cavity
ap = model.embed_operator(raising, position=0) 

bm1 = model.embed_operator(lowering, position=2) # position=2 is vib 1
bp1 = model.embed_operator(raising, position=2) 

bm2 = model.embed_operator(lowering, position=4) # position=4 is vib 2
bp2 = model.embed_operator(raising, position=4) 

sp1 = model.embed_operator(sigma_plus, position=1) # position=1 is qubit 1
sm1 = model.embed_operator(sigma_minus, position=1) 
sz1 = model.embed_operator(sigma_z, position=1)

sp2 = model.embed_operator(sigma_plus, position=3) # position=3 is qubit 2
sm2 = model.embed_operator(sigma_minus, position=3) 
sz2 = model.embed_operator(sigma_z, position=3)


\begin{equation}
H = \sum_i -\frac{\hbar \omega_i}{2} \sigma_{z,i} + \hbar \omega \hat{a}^\dagger \hat{a} +  \sum_i \hbar g_i ( \hat{a}  \sigma_{+,i} + \hat{a}^\dagger \sigma_{-,i} ) + \lambda \omega \sigma^+ \sigma^-( \hat{b}^{\dagger} + \hat{b} ) + \hbar \omega_v \hat{b}^{\dagger} \hat{b}
\end{equation}


In [5]:
# build Hamiltonian where qubit 1 is coupled to the cavity
H1 = -model.omega_qubit / 2 * sz1   # qubit 1 energy 
H1 -= model.omega_qubit / 2 * sz2   # qubit 2 energy
H1 += model.omega_cav * ap @ am     # cavity energy
H1 += model.omega_vib * bp1 @ bm1   # vib 1 energy
H1 += model.omega_vib * bp2 @ bm2   # vib 2 energy
H1 += model.g * (am @ sp1 + ap @ sm1) # qubit 1 - cavity coupling
H1 += model.omega_vib * model.lambda_vib * sp1 @ sm1 @ (bp1 + bm1) # qubit 1 - vib 1 coupling
H1 += model.omega_vib * model.lambda_vib * sp2 @ sm2 @ (bp2 + bm2) # qubit 2 - vib 2 coupling

# build Hamiltonian where qubit 2 is coupled to the cavity
H2 = np.copy(H1)
H2 -= model.g * (am @ sp1 + ap @ sm1)
H2 += model.g * (am @ sp2 + ap @ sm2)


H_un = np.copy(H2)
H2 -= model.g * (am @ sp2 + ap @ sm2)

In [15]:
N_state = 8
# diagonalize H1
eigenvalues_1, polariton_1 = np.linalg.eigh(H1)

# express H1 in polariton basis
Hpol1 = polariton_1.T @ H1 @ polariton_1

# compute expectation value of state |0,e,0,g,0> in uncoupled basis
psi = model.basis_state(N_state)
#print(psi)

E_exp_uncoupled = psi.T @ H1 @ psi

psi_pol = polariton_1[N_state,:].reshape(32,1)
#print(psi_pol)
E_exp_pol = psi_pol.T @ Hpol1 @ psi_pol

print(E_exp_uncoupled)
print(E_exp_pol)
#print(F"Expectation value in uncoupled basis is {E_exp_uncoupled:12.10f}")
#print(F"Expectation value in polariton basis is {E_exp_pol:12.10f}")




[[0.]]
[[-6.9388939e-17]]


In [ ]:
#target_basis_state = model.labels

model.represent_basis_in_eigenbasis(model.labels, polariton_1, energies=eigenvalues_1)

In [ ]:
# Get a basis vector
psi = model.basis_state(8)
print("Basis[8] =", model.labels[8], psi.T)

rho_init =  psi @ psi.T


In [ ]:
n_time_1 = np.pi / (4 * model.g)
n_time_2 = np.pi / (2 * model.g)

print(f"n_time_1 = {np.floor(n_time_1)}")
print(f"n_time_2 = {np.floor(n_time_2)}")




In [ ]:


dt = 0.01

n_time = int(40 / dt) # make sure we get to 40 a.u. of time

# arrays to store the populations at each time-step
pops_uncoupled = np.zeros((n_time,rho_init.shape[0]))

t = []


for i in range(n_time):
    if i * dt<n_time_1:
        # evolve in uncoupled basis
        rho_new = model.rk4_step(H1, rho_init, dt, hbar=1.0)

    elif i * dt < n_time_1 + n_time_2:
        rho_new = model.rk4_step(H2, rho_init, dt, hbar=1)

    else: 
        rho_new = model.rk4_step(H_un, rho_init, dt, hbar=1.0)


    t.append(i * dt)
    for k in range(rho_init.shape[0]):
        # store all populations
        pops_uncoupled[i,k] = np.real(rho_new[k,k])


    # copy updated rhos in each basis to the rho_init and rho_pol_init for next step
    rho_init = np.copy(rho_new)


In [ ]:
print(pops_uncoupled)

In [ ]:
from matplotlib import pyplot as plt
plt.plot(t, pops_uncoupled[:,8], label="$p_{99}$ uncoupled")
plt.plot(t, pops_uncoupled[:,16], label="$p_{1616}$ uncoupled")
plt.plot(t, pops_uncoupled[:,2], label="$p_{22}$ uncoupled")
plt.grid()
#plt.xlim(10,20)
plt.legend()
plt.show()